In [1]:
import pandas as pd
import os

# Set the directory containing the CSV files
directory = 'features'

# Get a list of all CSV files in the directory
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Initialize an empty data frame to store the combined data
combined_data = pd.DataFrame()

# Loop through each CSV file and append its data to the combined data frame
for csv_file in csv_files:
    file_path = os.path.join(directory, csv_file)
    data = pd.read_csv(file_path)
    combined_data = combined_data.append(data, ignore_index=True) # type: ignore

# Write the combined data frame to a new CSV file
combined_data.to_csv('features_combined.csv', index=False)

C:\Users\Mariam mohamed\AppData\Local\Temp\ipykernel_796\1322596109.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(data, ignore_index=True)


KeyboardInterrupt: 

In [2]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

def balance_data(X, y):
    """
    Balances a dataset by undersampling the majority class using the RandomUnderSampler algorithm.
    
    Parameters:
        X (numpy array): The feature data.
        y (numpy array): The target data.
        
    Returns:
        X_resampled (numpy array): The resampled feature data.
        y_resampled (numpy array): The resampled target data.
    """
    # Determine the class distribution
    class_counts = np.bincount(y)
    print(class_counts)
    
    # Determine the majority class label
    majority_class = np.argmax(class_counts) # type: ignore
    
    # Determine the number of samples to remove from the majority class
    n_samples = class_counts.min()
    
    # Undersample the majority class using the RandomUnderSampler algorithm
    rus = RandomUnderSampler(sampling_strategy={majority_class: n_samples}, random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X, y) # type: ignore
    
    return X_resampled, y_resampled

In [ ]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler

def balance_data_oversample(X, y):
    """
    Balances a dataset by oversampling the minority class using the RandomOverSampler algorithm.
    
    Parameters:
        X (numpy array): The feature data.
        y (numpy array): The target data.
        
    Returns:
        X_resampled (numpy array): The resampled feature data.
        y_resampled (numpy array): The resampled target data.
    """
    # Determine the class distribution
    class_counts = np.bincount(y)
    
    # Determine the minority class label
    minority_class = np.argmin(class_counts)
    
    # Determine the number of samples to add to the minority class
    n_samples = class_counts.max() - class_counts[minority_class]
    
    # Resample the minority class using the RandomOverSampler algorithm
    ros = RandomOverSampler(sampling_strategy={minority_class: n_samples}, random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X, y)
    
    return X_resampled, y_resampled

In [4]:
import pandas as pd
df = pd.read_csv('features_combined.csv')
X=df.drop(['id', 'start','end','target'], axis=1)
y = df['target']

In [4]:
balanced_x,balanced_y=balance_data(X,y)
balanced_y

[24668   130]


0      0
1      0
2      0
3      0
4      0
      ..
255    1
256    1
257    1
258    1
259    1
Name: target, Length: 260, dtype: int64

In [ ]:
# Combine X and y into a single data frame
data = pd.concat([balanced_x, balanced_y], axis=1)

# Write the data frame to a CSV file
data.to_csv('features_balanced.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

# Load the data
df = pd.read_csv('features_combined.csv')

# Define the features and target variables
X = df.drop(['id', 'start', 'end', 'target'], axis=1)
y = df['target']

# Split the data into training and testing sets using stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Check if there are at least 2 classes in the target variable
if len(set(y_train)) < 2 or len(set(y_test)) < 2:
    raise ValueError("The target variable must have at least 2 classes")

# Train an SVC model on the normal training data
model = SVC(kernel='rbf')
model.fit(X_train[y_train == 0], y_train[y_train == 0])

# Predict the class labels for the testing data
y_pred = model.predict(X_test)

# Evaluate the performance of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

ValueError: The number of classes has to be greater than one; got 1 class